In [ ]:
!pip install numpy matplotlib tensorflow pandas scikit-learn

In [ ]:
!wget https://github.com/Rav2/DNN_tutorial/raw/main/data.csv

In [ ]:
import tensorflow as tf

In [ ]:
import os, sys 
import numpy as np
import pandas as pd
#import tensorflow as tf
#from tensorflow import keras
#from keras import layers
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

# Load and preview the data

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.info() 

In [ ]:
df.head()

In [ ]:
df.describe()

This data is for the 1909.09226 analysis. First 9 columns contains yields in signal bins, then there are yields in 5 control regions and 4 negative likelihood values: expected with mu=0, expected with mu=1, observed with mu=0, observed with mu=1. 

Let's split data into training, validation and testing samples

In [ ]:
train, val, test = np.split(df.sample(frac=1),[int(.8*len(df)), int(.9*len(df))])
train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
columns = train.columns
std_scaler = StandardScaler()
train_scaled = std_scaler.fit_transform(train.iloc[:, :-4].to_numpy())
mean_arr = std_scaler.mean_ 
std_arr = np.sqrt(std_scaler.var_)
train_scaled = pd.DataFrame(train_scaled, columns=columns[:-4])
val_scaled = std_scaler.transform(val.iloc[:, :-4].to_numpy())
val_scaled = pd.DataFrame(val_scaled, columns=columns[:-4])
test_scaled = std_scaler.transform(test.iloc[:, :-4].to_numpy())
test_scaled = pd.DataFrame(test_scaled, columns=columns[:-4])


for col in columns[-4:]:
    train_scaled.loc[:, col] = train.loc[:, col]
    test_scaled.loc[:, col] = test.loc[:, col]
    val_scaled.loc[:, col] = val.loc[:, col]


# Building model

In [ ]:
def get_model(inputs, neurons = 256, blocks=4, l2=1e-3):
    input_layer = keras.Input(shape=(inputs,))
    xx = input_layer
    for ii in range(blocks):
        xx = keras.layers.Dense(neurons, activation=None, kernel_regularizer=keras.regularizers.L2(l2))(xx)
        xx = keras.layers.BatchNormalization()(xx)
        xx = keras.layers.Activation(tf.nn.relu)(xx)
    xx = keras.layers.Dense(4, activation="ReLU",)(xx)
    model = keras.Model(input_layer, xx) 
    return model

In [ ]:
model = get_model(len(columns)-4, 32, 3, 1e-3)

In [ ]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-4), metrics=['mae','mape'])

In [ ]:
print(model.summary())

# Train model

In [ ]:
history = model.fit(
            train_scaled.iloc[:,:-4], train_scaled.iloc[:, -4:],
            batch_size=250, epochs=50, 
            validation_data=[val_scaled.iloc[:,:-4], val_scaled.iloc[:, -4:]]
                 )

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val', alpha=0.9)
plt.yscale('log')
plt.xlabel('epoch')
plt.ylabel('MSE loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['mae'], label='train')
plt.plot(history.history['val_mae'], label='val', alpha=0.9)
plt.yscale('log')
plt.xlabel('epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['mape'], label='train')
plt.plot(history.history['val_mape'], label='val', alpha=0.9)
plt.yscale('log')
plt.xlabel('epoch')
plt.ylabel('MAPE')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_scaled.iloc[:, :-4], test_scaled.iloc[:, -4:], batch_size=128)

In [ ]:
y_true = test_scaled.iloc[:, -4:]
y_pred = model.predict(test_scaled.iloc[:, :-4])
mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)

In [ ]:
print(y_true.shape, y_pred.shape, mape.shape)

In [ ]:
y_true

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10,4))
for ll in range(2):
    axs[ll].hist2d(y_true.iloc[:, ll*2+1], mape,  50, norm=mpl.colors.LogNorm(vmin=1, vmax=1e2))
    axs[ll].set_xlabel(columns[14+ll*2+1])
    axs[ll].set_ylabel('MAPE')
plt.tight_layout()
plt.show()

# Exercises

* Experiment with neural network hyperparameters to improve MAPE results
* Make 2D histograms of -2*log[ L_mu1/Lm_u0] for both expected and observed